In [1]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import to_int, ints, flatten, _flatten, rev, zippify, list_multiply
from aocutils.grid import arr_to_dict, grid_to_dict, neighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d
from aocutils.maze import bfs, dijkstra, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, lis
from aocutils.special import md5, binarysearch, deduce_matches, find_pattern_in_iter, find_repeat, find_cycle
from aocutils.visuals import visualize_graph, labelize, animate_grid
from aocutils.datascience import untar_data, timeit, load, save, nan_inspect
from aocutils.cfg import CFG
from aocutils.shunting import ShuntingYard

In [80]:
f=open('input.txt')
lines = [line.rstrip('\n') for line in f]

# print amount of rows and first 100 character of first line
len(lines), lines[0][:100]

(231, 'bot 171 gives low to bot 4 and high to bot 84')

In [88]:
vals = defaultdict(lambda: [])
give = defaultdict(lambda: [])
outputs = defaultdict(list)
for line in lines:
    if line.startswith('bot'):
        _, bot, _, _, _, type1, nr1, _, _, _, type2, nr2 = line.split()
        give[int(bot)] = ((type1, int(nr1)), (type2, int(nr2)))
    elif line.startswith('value'):
        _, nr1, _, _, type1, n1 = line.split()
        vals[int(n1)].append(int(nr1))

tocheck = deque([k for k,v in vals.items() if len(v)==2]) 
seen = set()
mapping = {'bot': vals, 'output': outputs}
while tocheck:
    for _ in range(len(tocheck)):
        cur = tocheck.popleft()
        seen.add(cur)
        low, high = sorted(vals[cur])
        (type1, nr1), (type2, nr2) = give[cur]
        mapping[type1][nr1].append(low)
        mapping[type2][nr2].append(high)
    for k,v in vals.items():
        if len(v) == 2 and k not in seen:
            tocheck.append(k)
        

In [ ]:
for k, v in vals.items():
    if 17 in v and 61 in v:
        print(k,v)
outputs[0][0] * outputs[1][0] * outputs[2][0]      
        

In [95]:
import random 
# Zobrist hashing
kinds = { "Thulium": 0, "Plut": 0, "Stront": 0, "Prom": 0, "Ruth": 0, "H": 0, "Li": 0, "El": 0, "Di": 0 }
cgh = {"Chip": 0, "Gen": 0}
floorh = {0: 0, 1: 0, 2: 0, 3: 0 }
matchedpairs = {}

for k in kinds.keys():
    kinds[k] = random.randint(0, 99999999999999999999999999)
for k in cgh.keys():
    cgh[k] = random.randint(0, 99999999999999999999999999)
for k in floorh.keys():
    floorh[k] = random.randint(0, 99999999999999999999999999)

for k in range(len(kinds)):
    kinds[k] = random.randint(0, 99999999999999999999999999)
for k in (0, 1):
    cgh[k] = random.randint(0, 99999999999999999999999999)
for k in range(10):
    matchedpairs[k] = random.randint(0, 99999999999999999999999999)   

# Use Zobrist hashing to avoid visiting repeated states
visited = set()
def shash(state):
    res = 0
    for i, f in enumerate(state[1]):
        gens = set()
        chips = set()
        for cg, kind in f:
            if cg == 1:
                gens.add(kind)
            else:
                chips.add(kind)

        mps = gens.intersection(chips)                      
        gens = gens - mps                                   
        chips = chips - mps                                  

        for kind in chips:
            res = res ^ (floorh[i] * cgh[0] * kinds[kind])
        for kind in gens:
            res = res ^ (floorh[i] * cgh[1] * kinds[kind])
        res = res ^ (floorh[i] * matchedpairs[len(mps)])         

    res = res ^ (floorh[ state[2] ] )
    return res